### Defintion

Going Modular: turning notebook code into a series of different Python scripts that offer similar funcionality.

THe main concept of this section is: **turn useful notebook code cells into reusable Python files.**

Going from Jupyter notebooks where you can visualize nad run quick experiment to then save the most useful parts into py scripts.



### Running code from a Py srcipt

You might have to run the code like this:

`python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS`

`train.py` is the target Python script

`--model`, `--batch_size`, `--lr` and `--num_epochs` are known as argument flags.

For example, let's say we wanted to train our TinyVGG model from notebook 04 for 10 epochs with a batch size of 32 and a learning rate of 0.001:

`python train.py --model tinyvgg --batch_size 32 --lr 0.001 --num_epochs 10`


### Things to note

**Docstrings** - writing reproducible and understnadable code is importatn and with this in mind, each of the funtions/classes we'll be putting into stripts has been created with Google's Python docstring style in mind.

**Imports at the top of scripts** - Sicne all of the Python scripts we are going to create could be considered a small rpgram on thier own, all of the sripts requre thier input modeuls be imported at the stat of the script.

## 0. Cell mode vs. script mode

**Cell mode** --> notebook run normally each cell in the notebook is either code or markdown(cell version of this notebook can be seen in 04_pytorch.ipynb)

**Script mode** --> many of teh code cells may be turned into Python scripts

## 1. Get Data

In [3]:
import os
import requests
import zipfile
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
  print(f"{image_path} is a diretory")
else:
  print(f"Downloading...")
  image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  f.write(request.content) # write the requested conctnet to the file

with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
  zip_ref.extractall(image_path)

os.remove(data_path/ "pizza_steak_sushi.zip")

data/pizza_steak_sushi is a diretory


data/└── pizza_steak_sushi/
    ├── train/
    │   ├── pizza/
    │   │   ├── train_image01.jpeg
    │   │   ├── test_image02.jpeg
    │   │   └── ...
    │   ├── steak/
    │   │   └── ...
    │   └── sushi/
    │       └── ...
    └── test/
        ├── pizza/
        │   ├── test_image01.jpeg
        │   └── test_image02.jpeg
        ├── steak/
        └── sushi/

In [13]:
# Create a directory going_modular scripts
import os
os.makedirs("going_modular")

## 2. Create Datasets and DataLoaders

In [38]:
%%writefile going_modular/data_setup.py

'''
Contains Functionality for creating PyTorch DataLoaders for image classification data.
'''

import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  ''' Create training and testing DataLoaders

  Takes in a trainig dir adn testing dir path and
  turns them into PyTorch DataSets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transfomrs to perform on trainign and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_name).
    Example usage:
        rain_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  '''

  # Use ImageFolder to create dataset
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get calss names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Overwriting going_modular/data_setup.py


## 3. Making a model(TinyVGG)

In [44]:
%%writefile going_modular/model_builder.py
''' Contain PyTorch model code to instantiate a TinyVGG model. '''

import torch
from torch import nn

class TinyVGG(nn.Module):
  ''' Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer webiste in PyTorch

  Args:
    input_shapes: An integer indicating number of input channels
    hidden_units: AN integer indicating number of hidden units betwwn layers.
    output_shapes: An integer indicating number of output units.
  '''

  def __init__(self, input_shapes: int,
               hidden_units: int,
               output_shapes: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(
            in_channels=input_shapes,
            out_channels=hidden_units,
            kernel_size=3,
            stride=1,
            padding=0
            ),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0
                  ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(
            in_channels=hidden_units,
            out_channels=hidden_units,
            kernel_size=3,
            stride=1,
            padding=0
            ),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0
                  ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13, # spital dim
                  out_features=output_shapes)
    )

  def forward(self, x):
    return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Overwriting going_modular/model_builder.py


## 4. Creating `train_step()` and `test_step()` functions and `train()` to combine them

In [18]:
%%writefile going_modular/engine.py

"""
Contains functions for training and testing a PyTorch model.
"""
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1000, 0.8000)
  """
  # train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # Forward pass
      y_pred = model(X)

      # Calculate and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # Optimizer zero grad
      optimizer.zero_grad()

      # Loss backward
      loss.backward()

      # Optimizer step
      optimizer.step()

      # ACC
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1) # logits --> probs --> labels
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.2000, 0.8000)
  """
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # Forward pass
          test_pred_logits = model(X)

          # Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # average loss and accuracy per batch
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  '''Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
  '''
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
                                      dataloader=test_dataloader,
                                      loss_fn=loss_fn,
                                      device=device)

      # Print out results
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

Writing going_modular/engine.py


## 5. Creating a function to save the model

It is common to store helper functions in a file called `utils.py`

In [19]:
%%writefile going_modular/utils.py

''' Contains verious utility functions for PyTorch model trainign and saving. '''
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name:str):
  ''' Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include either ".pth" or ".pt" as the file extenstion.

  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  '''
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save pth
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.plt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f" Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing going_modular/utils.py


## 6. Train, evaluate and save the model

In [40]:
%%writefile going_modular/train.py
''' Trains a PyTorch image classification omdel using device-agnostic code. '''

import os
import torch
# Since train.py is going to be inside going_modular directory, we can import the other modules via import without from X import ...
import data_setup, engine, model_builder, utils
from torchvision import transforms

# setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LR = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# device target
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create trsnfomrs
data_transform = transforms.Compose(
    [
        transforms.Resize((64,64)),
        transforms.ToTensor()
    ]
)

# Create dataloaders with the help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with hel form model_buidler.py
model = model_builder.TinyVGG(
    input_shapes=3,
    hidden_units=HIDDEN_UNITS,
    output_shapes=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(),
                             lr=LR)

# Start training with help form engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="modular_model_tinvgg.pth")


Overwriting going_modular/train.py


In [45]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1053 | train_acc: 0.2891 | test_loss: 1.0986 | test_acc: 0.2604
 20% 1/5 [00:01<00:07,  1.92s/it]Epoch: 2 | train_loss: 1.0918 | train_acc: 0.4141 | test_loss: 1.0750 | test_acc: 0.5417
 40% 2/5 [00:03<00:05,  1.82s/it]Epoch: 3 | train_loss: 1.1133 | train_acc: 0.2812 | test_loss: 1.0655 | test_acc: 0.5417
 60% 3/5 [00:05<00:03,  1.94s/it]Epoch: 4 | train_loss: 1.0923 | train_acc: 0.4023 | test_loss: 1.0818 | test_acc: 0.5417
 80% 4/5 [00:08<00:02,  2.40s/it]Epoch: 5 | train_loss: 1.0905 | train_acc: 0.4023 | test_loss: 1.0744 | test_acc: 0.5417
100% 5/5 [00:10<00:00,  2.19s/it]
 Saving model to: models/modular_model_tinvgg.pth
